In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from time import strftime, localtime
TCT = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Company_Tweet.csv\Company_Tweet.csv", index_col="tweet_id")
TT  = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv", index_col="tweet_id")

In [3]:
TT["body"]

tweet_id
550441509175443456     lx21 made $10,008  on $AAPL -Check it out! htt...
550441672312512512     Insanity of today weirdo massive selling. $aap...
550441732014223360     S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...
550442977802207232     $GM $TSLA: Volkswagen Pushes 2014 Record Recal...
550443807834402816     Swing Trading: Up To 8.91% Return In 14 Days h...
                                             ...                        
1212159765914079234    That $SPY $SPX puuump in the last hour was the...
1212159838882533376    In 2020 I may start Tweeting out positive news...
1212160015332728833    Patiently Waiting for the no twitter sitter tw...
1212160410692046849    I don't discriminate. I own both $aapl and $ms...
1212160477159206912    $AAPL #patent 10,522,475 Vertical interconnect...
Name: body, Length: 3717964, dtype: object

In [4]:
# Load the regular expression library
import re
TT = TT[:1000]
# Remove punctuation
TT['body_processed'] = TT['body'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
TT['body_processed'] = TT['body_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
TT['body_processed'].head()


<>:5: DeprecationWarning: invalid escape sequence '\.'
<>:5: DeprecationWarning: invalid escape sequence '\.'
C:\Users\Fabio\AppData\Local\Temp\ipykernel_16356\236631969.py:5: DeprecationWarning: invalid escape sequence '\.'
  TT['body_processed'] = TT['body'].map(lambda x: re.sub('[,\.!?]', '', x))


tweet_id
550441509175443456    lx21 made $10008  on $aapl -check it out http:...
550441672312512512    insanity of today weirdo massive selling $aapl...
550441732014223360    s&p100 #stocks performance $hd $low $sbux $tgt...
550442977802207232    $gm $tsla: volkswagen pushes 2014 record recal...
550443807834402816    swing trading: up to 891% return in 14 days ht...
Name: body_processed, dtype: object

In [5]:
# Import the wordcloud library
import time
from wordcloud import WordCloud
time.strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# Join the different processed titles together.
long_string = ','.join(list(TT['body_processed'].values))
time.strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')
time.strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# Generate a word cloud
wordcloud.generate(long_string)
time.strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# Visualize the word cloud
#wordcloud.to_image()

c:\Users\Fabio\AppData\Local\Programs\Python\Python311\Lib\site-packages\wordcloud\wordcloud.py:106: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  self.colormap = plt.cm.get_cmap(colormap)


'2012-09-13 08:22:50'

In [6]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]


data = TT['body_processed'].values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['lx', 'made', 'aapl', 'check', 'http', 'profitly', 'mnd', 'saff', 'learn', 'howtotrade', 'http', 'bitly', 'nljx', 'exe', 'watt', 'imrs', 'cach', 'gmo']


In [7]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]


In [9]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.039*"http" + 0.038*"aapl" + 0.018*"goog" + 0.014*"tsla" + 0.012*"stock" + '
  '0.010*"apple" + 0.009*"inc" + 0.009*"google" + 0.009*"bitly" + '
  '0.008*"updated"'),
 (1,
  '0.052*"http" + 0.030*"aapl" + 0.016*"amzn" + 0.013*"tsla" + 0.012*"top" + '
  '0.011*"googl" + 0.010*"stocks" + 0.009*"fb" + 0.009*"new" + 0.009*"year"'),
 (2,
  '0.043*"http" + 0.039*"aapl" + 0.023*"apple" + 0.008*"goog" + 0.008*"amzn" + '
  '0.008*"price" + 0.008*"msft" + 0.007*"tsla" + 0.006*"stock" + 0.006*"fb"'),
 (3,
  '0.048*"http" + 0.042*"aapl" + 0.020*"apple" + 0.019*"amzn" + 0.015*"stock" '
  '+ 0.012*"tsla" + 0.009*"year" + 0.007*"stocks" + 0.007*"fb" + '
  '0.007*"googl"'),
 (4,
  '0.048*"http" + 0.034*"aapl" + 0.028*"amzn" + 0.016*"year" + 0.015*"amazon" '
  '+ 0.015*"onwsjcom" + 0.014*"billion" + 0.014*"since" + 0.014*"jeff" + '
  '0.013*"worst"'),
 (5,
  '0.046*"http" + 0.032*"aapl" + 0.016*"apple" + 0.011*"goog" + 0.011*"bitly" '
  '+ 0.010*"news" + 0.010*"google" + 0.009*"article" + 0.00

In [21]:
import os
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle 


# Visualize the topics
pyLDAvis.enable_notebook()

pyLDAvis_path = "C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Final Project\\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\\pyLDAvis\\"

#LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
LDAvis_data_filepath = os.path.join(pyLDAvis_path, 'ldavis_prepared_'+str(num_topics))


# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(num_topics) +'.html')

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.125894  0.070191       1        1  14.162494
3     -0.011969 -0.052809       2        1  13.355756
9      0.016915 -0.095325       3        1  13.231007
8      0.053282 -0.017200       4        1  11.672795
5      0.096719  0.115203       5        1  10.200289
0      0.079275  0.017866       6        1   9.278959
1     -0.076304  0.018604       7        1   8.748591
2      0.052012 -0.054438       8        1   7.794147
6     -0.097774 -0.022012       9        1   6.921973
7      0.013738  0.019919      10        1   4.633989, topic_info=        Term       Freq       Total Category  logprob  loglift
210  billion  41.000000   41.000000  Default  30.0000  30.0000
212     lost  38.000000   38.000000  Default  29.0000  29.0000
211     jeff  37.000000   37.000000  Default  28.0000  28.0000
209    bezos  41.000000   41.000000  Default  27.0000  27.0000
214    worst  42.000000   42.000000  Default  26.0000  26.0000
..       ...        ...         ...      ...      ...      ...
95       new   2.142585   46.082056  Topic10  -5.6581   0.0033
79       end   1.764535   12.141306  Topic10  -5.8523   1.1430
90     apple   2.280440  178.940024  Topic10  -5.5958  -1.2909
447   iphone   1.856900   39.182355  Topic10  -5.8012   0.0224
852     musk   1.605708   10.055723  Topic10  -5.9466   1.2372

[712 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1364      4  0.841005      aa
0         1  0.130664    aapl
0         2  0.151080    aapl
0         3  0.189871    aapl
0         4  0.138830    aapl
...     ...       ...     ...
1499      2  0.252362    ztec
1499      4  0.757085    ztec
2388      8  0.639319  zxkbma
2536      4  0.503704     zya
2536      5  0.251852     zya

[1405 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 10, 9, 6, 1, 2, 3, 7, 8])